### Serverless Monitoring and Budget Alert Queries
#### Based on Databricks documentation for tracking serverless spend

In [0]:
%sql
-- Alert: Workspace spend exceeding threshold in last 30 days
CREATE MATERIALIZED VIEW workspace_spend_alerts AS
SELECT
   usage_data.workspace_id,
   SUM(usage_data.usage_quantity * pricing_data.pricing.default) as list_cost,
   SUM(usage_data.usage_quantity * pricing_data.pricing.effective_list.default) as effective_cost,
   SUM(CASE WHEN usage_data.usage_date >= CURRENT_DATE() - INTERVAL 7 DAYS THEN usage_data.usage_quantity * pricing_data.pricing.default ELSE 0 END) as t7d_list_cost,
   SUM(CASE WHEN usage_data.usage_date >= CURRENT_DATE() - INTERVAL 7 DAYS THEN usage_data.usage_quantity * pricing_data.pricing.effective_list.default ELSE 0 END) as t7d_effective_cost,
   SUM(CASE WHEN usage_data.usage_date >= CURRENT_DATE() - INTERVAL 14 DAYS AND usage_data.usage_date < CURRENT_DATE() - INTERVAL 7 DAYS THEN usage_data.usage_quantity * pricing_data.pricing.default ELSE 0 END) as t14d_list_cost,
   SUM(CASE WHEN usage_data.usage_date >= CURRENT_DATE() - INTERVAL 14 DAYS AND usage_data.usage_date < CURRENT_DATE() - INTERVAL 7 DAYS THEN usage_data.usage_quantity * pricing_data.pricing.effective_list.default ELSE 0 END) as t14d_effective_cost
FROM system.billing.usage usage_data
INNER JOIN system.billing.list_prices pricing_data on
   usage_data.cloud = pricing_data.cloud and
   usage_data.sku_name = pricing_data.sku_name and
   usage_data.usage_start_time >= pricing_data.price_start_time and
   (usage_data.usage_end_time <= pricing_data.price_end_time or pricing_data.price_end_time is null)
WHERE
   usage_data.sku_name LIKE '%SERVERLESS%'
   AND usage_data.billing_origin_product IN ("JOBS", "INTERACTIVE")
   AND usage_data.usage_date >= CURRENT_DATE() - INTERVAL 30 DAYS
GROUP BY
   usage_data.workspace_id
ORDER BY
   effective_cost DESC;

In [0]:
%sql
-- Alert: User spend exceeding threshold in last 30 days
CREATE MATERIALIZED VIEW user_spend_alerts AS
SELECT
   usage_data.identity_metadata.run_as,
   SUM(usage_data.usage_quantity * pricing_data.pricing.default) as list_cost,
   SUM(usage_data.usage_quantity * pricing_data.pricing.effective_list.default) as effective_cost,
   SUM(CASE WHEN usage_data.usage_date >= CURRENT_DATE() - INTERVAL 7 DAYS THEN usage_data.usage_quantity * pricing_data.pricing.default ELSE 0 END) as t7d_list_cost,
   SUM(CASE WHEN usage_data.usage_date >= CURRENT_DATE() - INTERVAL 7 DAYS THEN usage_data.usage_quantity * pricing_data.pricing.effective_list.default ELSE 0 END) as t7d_effective_cost,
   SUM(CASE WHEN usage_data.usage_date >= CURRENT_DATE() - INTERVAL 14 DAYS AND usage_data.usage_date < CURRENT_DATE() - INTERVAL 7 DAYS THEN usage_data.usage_quantity * pricing_data.pricing.default ELSE 0 END) as t14d_list_cost,
   SUM(CASE WHEN usage_data.usage_date >= CURRENT_DATE() - INTERVAL 14 DAYS AND usage_data.usage_date < CURRENT_DATE() - INTERVAL 7 DAYS THEN usage_data.usage_quantity * pricing_data.pricing.effective_list.default ELSE 0 END) as t14d_effective_cost
FROM system.billing.usage usage_data
INNER JOIN system.billing.list_prices pricing_data on
   usage_data.cloud = pricing_data.cloud and
   usage_data.sku_name = pricing_data.sku_name and
   usage_data.usage_start_time >= pricing_data.price_start_time and
   (usage_data.usage_end_time <= pricing_data.price_end_time or pricing_data.price_end_time is null)
WHERE
   usage_data.sku_name LIKE '%SERVERLESS%'
   AND usage_data.billing_origin_product IN ("JOBS", "INTERACTIVE")
   AND usage_data.usage_date >= CURRENT_DATE() - INTERVAL 30 DAYS
GROUP BY
   usage_data.identity_metadata.run_as
ORDER BY
   effective_cost DESC;

In [0]:
%sql
-- Alert: Job spend exceeding threshold in last 30 days
CREATE MATERIALIZED VIEW job_spend_alerts AS
SELECT
   usage_data.workspace_id,
   usage_data.usage_metadata.job_id,
   SUM(usage_data.usage_quantity * pricing_data.pricing.default) as list_cost,
   SUM(usage_data.usage_quantity * pricing_data.pricing.effective_list.default) as effective_cost
FROM system.billing.usage usage_data
INNER JOIN system.billing.list_prices pricing_data on
   usage_data.cloud = pricing_data.cloud and
   usage_data.sku_name = pricing_data.sku_name and
   usage_data.usage_start_time >= pricing_data.price_start_time and
   (usage_data.usage_end_time <= pricing_data.price_end_time or pricing_data.price_end_time is null)
WHERE
   usage_data.sku_name LIKE '%SERVERLESS%'
   AND usage_data.billing_origin_product IN ("JOBS")
   AND usage_data.usage_date >= CURRENT_DATE() - INTERVAL 30 DAYS
GROUP BY
   usage_data.workspace_id, usage_data.usage_metadata.job_id
ORDER BY
   effective_cost DESC;

In [0]:
%sql
-- Serverless notebooks sorted by total spend
CREATE MATERIALIZED VIEW serverless_notebook_spend AS
SELECT
  usage_data.usage_metadata.notebook_id,
  usage_data.usage_metadata.notebook_path,
  usage_data.identity_metadata.run_as,
  SUM(usage_data.usage_quantity) as total_dbu,
  SUM(usage_data.usage_quantity * pricing_data.pricing.default) as list_cost,
  SUM(usage_data.usage_quantity * pricing_data.pricing.effective_list.default) as effective_cost
FROM
  system.billing.usage usage_data
INNER JOIN system.billing.list_prices pricing_data on
  usage_data.cloud = pricing_data.cloud and
  usage_data.sku_name = pricing_data.sku_name and
  usage_data.usage_start_time >= pricing_data.price_start_time and
  (usage_data.usage_end_time <= pricing_data.price_end_time or pricing_data.price_end_time is null)
WHERE
  usage_data.usage_metadata.notebook_id is not null
  and usage_data.billing_origin_product = 'INTERACTIVE'
  and usage_data.product_features.is_serverless
  and usage_data.usage_unit = 'DBU'
  and usage_data.usage_date >= DATEADD(day, -30, current_date)
GROUP BY
  usage_data.usage_metadata.notebook_id,
  usage_data.usage_metadata.notebook_path,
  usage_data.identity_metadata.run_as
ORDER BY
  effective_cost DESC;

In [0]:
%sql
-- Serverless jobs sorted by total spend
CREATE MATERIALIZED VIEW serverless_job_spend AS
SELECT
  usage_data.usage_metadata.job_id,
  usage_data.usage_metadata.job_name,
  SUM(usage_data.usage_quantity) as total_dbu,
  SUM(usage_data.usage_quantity * pricing_data.pricing.default) as list_cost,
  SUM(usage_data.usage_quantity * pricing_data.pricing.effective_list.default) as effective_cost
FROM
  system.billing.usage usage_data
INNER JOIN system.billing.list_prices pricing_data on
  usage_data.cloud = pricing_data.cloud and
  usage_data.sku_name = pricing_data.sku_name and
  usage_data.usage_start_time >= pricing_data.price_start_time and
  (usage_data.usage_end_time <= pricing_data.price_end_time or pricing_data.price_end_time is null)
WHERE
  usage_data.usage_metadata.job_id is not null
  and usage_data.usage_unit = 'DBU'
  and usage_data.usage_date >= DATEADD(day, -30, current_date)
  and usage_data.sku_name like '%JOBS_SERVERLESS_COMPUTE%'
GROUP BY
  usage_data.usage_metadata.job_id,
  usage_data.usage_metadata.job_name
ORDER BY
  effective_cost DESC;

In [0]:
%sql
-- Cost consumed by specific user across notebooks and jobs
CREATE MATERIALIZED VIEW user_serverless_consumption AS
SELECT
  usage_data.usage_metadata.job_id,
  usage_data.usage_metadata.job_name,
  usage_data.usage_metadata.notebook_id,
  usage_data.usage_metadata.notebook_path,
  usage_data.identity_metadata.run_as,
  SUM(usage_data.usage_quantity) as total_dbu,
  SUM(usage_data.usage_quantity * pricing_data.pricing.default) as list_cost,
  SUM(usage_data.usage_quantity * pricing_data.pricing.effective_list.default) as effective_cost
FROM
  system.billing.usage usage_data
INNER JOIN system.billing.list_prices pricing_data on
  usage_data.cloud = pricing_data.cloud and
  usage_data.sku_name = pricing_data.sku_name and
  usage_data.usage_start_time >= pricing_data.price_start_time and
  (usage_data.usage_end_time <= pricing_data.price_end_time or pricing_data.price_end_time is null)
WHERE
  usage_data.billing_origin_product in ('JOBS','INTERACTIVE')
  and usage_data.product_features.is_serverless
  and usage_data.usage_unit = 'DBU'
  and usage_data.usage_date >= DATEADD(day, -30, current_date)
GROUP BY
  ALL
ORDER BY
  effective_cost DESC;

In [0]:
%sql
-- Serverless compute cost by custom tag
CREATE MATERIALIZED VIEW serverless_consumption_by_tag AS
SELECT
  usage_data.usage_metadata.job_id,
  usage_data.usage_metadata.job_name,
  usage_data.usage_metadata.notebook_id,
  usage_data.usage_metadata.notebook_path,
  usage_data.custom_tags,
  SUM(usage_data.usage_quantity) as total_dbu,
  SUM(usage_data.usage_quantity * pricing_data.pricing.default) as list_cost,
  SUM(usage_data.usage_quantity * pricing_data.pricing.effective_list.default) as effective_cost
FROM
  system.billing.usage usage_data
INNER JOIN system.billing.list_prices pricing_data on
  usage_data.cloud = pricing_data.cloud and
  usage_data.sku_name = pricing_data.sku_name and
  usage_data.usage_start_time >= pricing_data.price_start_time and
  (usage_data.usage_end_time <= pricing_data.price_end_time or pricing_data.price_end_time is null)
WHERE
  usage_data.billing_origin_product in ('JOBS','INTERACTIVE')
  and usage_data.product_features.is_serverless
  and usage_data.usage_unit = 'DBU'
  and usage_data.usage_date >= DATEADD(day, -30, current_date)
GROUP BY
  ALL
ORDER BY
  effective_cost DESC;